In [ ]:
!pip install numpy scipy tifffile imagecodecs


In [ ]:


# ----------------------------------------------------
# Step 2: Import libraries
# ----------------------------------------------------
import tifffile
import numpy as np
from scipy.io import wavfile
# from google.colab import files # For uploading/downloading if needed

# ----------------------------------------------------
# Step 3: Define the sonification functions
# ----------------------------------------------------
def generate_sine_wave(frequency, duration, sample_rate, amplitude):
    """Generates a sine wave."""
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = amplitude * np.sin(2 * np.pi * frequency * t)
    return wave

def sonify_3channel_tiff(
    tiff_path,
    output_wav_path="output_sonification.wav",
    scan_duration_per_column=0.05,  # Seconds each image column will represent in sound
    sample_rate=44100,
    base_frequencies=(220, 330, 440),  # Hz for Ch1, Ch2, Ch3 (e.g., A3, E4, A4)
    master_volume=0.8, # General volume control (0.0 to 1.0)
    output_channels=3 # 3 for 3-channel audio, 2 for stereo (mixdown/selection)
):
    """
    Generates sound from a 3-channel TIFF image.
    Each image channel modulates the amplitude of a sine wave in a corresponding audio channel.
    The image is scanned column by column.
    """
    try:
        image = tifffile.imread(tiff_path)
    except Exception as e:
        print(f"Error reading TIFF file '{tiff_path}': {e}")
        return

    if image.ndim == 2:
        print("Image is grayscale. Converting to 3-channel by duplicating.")
        image = np.stack([image, image, image], axis=-1)
    elif image.ndim == 3 and image.shape[-1] != 3:
        if image.shape[-1] == 1: # (H, W, 1)
             print("Image has 1 channel. Converting to 3-channel by duplicating.")
             image = np.concatenate([image, image, image], axis=-1)
        elif image.shape[-1] > 3:
            print(f"Image has {image.shape[-1]} channels. Using only the first 3.")
            image = image[:, :, :3]
        else: # e.g. 2 channels
            print(f"Image has {image.shape[-1]} channels. Attempting to pad to 3 for sonification.")
            if image.shape[-1] == 2:
                zeros_channel = np.zeros_like(image[:,:,0:1])
                image = np.concatenate([image, zeros_channel], axis=-1)
            else: # Should not happen if previous checks are correct
                 print(f"Unexpected number of channels after initial checks: {image.shape[-1]}. Using first available.")
                 image = image[:,:,:min(3, image.shape[-1])] # Take up to 3, pad later if needed

    elif image.ndim != 3 or image.shape[-1] < 1 : # Check if last dim is channels
        # Try to guess if channels are the first dimension
        if image.ndim == 3 and image.shape[0] >= 1 and image.shape[0] <=4 : # common channel counts
            print(f"Assuming channels are the first dimension (shape {image.shape}), transposing to (height, width, channels).")
            image = np.transpose(image, (1, 2, 0)) # from (C, H, W) to (H, W, C)
            if image.shape[-1] > 3:
                print(f"Image has {image.shape[-1]} channels after transpose. Using only the first 3.")
                image = image[:, :, :3]
            elif image.shape[-1] == 1:
                print("Transposed image has 1 channel. Converting to 3-channel by duplicating.")
                image = np.concatenate([image, image, image], axis=-1)
            elif image.shape[-1] == 2:
                print("Transposed image has 2 channels. Padding with a silent third channel.")
                zeros_channel = np.zeros_like(image[:,:,0:1])
                image = np.concatenate([image, zeros_channel], axis=-1)
        else:
            print(f"Error: Image has an unexpected shape: {image.shape}. Expected HxWx3 or CxHxW (with C=3).")
            return

    # Final check for 3 channels after processing
    if image.shape[-1] != 3:
        print(f"Error: After processing, image does not have 3 channels (shape: {image.shape}). Cannot proceed with 3-channel sonification as configured.")
        return

    height, width, num_channels = image.shape # Should be 3 now
    print(f"Image loaded: {width}x{height} pixels, {num_channels} channels.")
    print(f"Image dtype: {image.dtype}, min: {image.min()}, max: {image.max()}")

    # Normalize image data to 0-1 range based on its data type
    if np.issubdtype(image.dtype, np.integer):
        if image.dtype == np.uint8:
            max_val = 255.0
        elif image.dtype == np.uint16:
            max_val = 65535.0
        elif image.dtype == np.int8: # Though less common for images
            max_val = 127.0
            image = image.astype(np.float32) + 128.0 # shift to positive range before normalization
        elif image.dtype == np.int16: # Though less common for images
            max_val = 32767.0
            image = image.astype(np.float32) + 32768.0 # shift to positive range
        else: # Other integer types
            type_info = np.iinfo(image.dtype)
            max_val = float(type_info.max)
            if type_info.min < 0: # If signed, shift to be all positive before scaling
                image = image.astype(np.float64) - type_info.min
                max_val -= type_info.min
    elif np.issubdtype(image.dtype, np.floating):
        # For float, assume it's either already 0-1 or scale by its own max
        img_max_val = image.max()
        img_min_val = image.min()
        if img_max_val > 1.0 or img_min_val < 0.0: # Needs normalization
            if img_max_val == img_min_val: # Flat image
                max_val = 1.0
                image = np.zeros_like(image, dtype=np.float32) # Normalize to 0
            else:
                image = (image.astype(np.float32) - img_min_val) / (img_max_val - img_min_val)
                max_val = 1.0 # Already normalized to 0-1
        else: # Already in 0-1 range (presumably)
            max_val = 1.0
    else:
        print(f"Warning: Unsupported image data type {image.dtype}. Attempting to scale by image.max().")
        max_val = image.max()

    if max_val == 0: max_val = 1.0 # Avoid division by zero for blank images
    normalized_image = image.astype(np.float32) / max_val
    normalized_image = np.clip(normalized_image, 0.0, 1.0) # Ensure it's strictly 0-1

    # --- Audio Generation ---
    audio_channels_data = [[] for _ in range(3)]

    for x in range(width):
        for ch_idx in range(3):
            column_data = normalized_image[:, x, ch_idx]
            avg_intensity = np.mean(column_data)

            segment = generate_sine_wave(
                frequency=base_frequencies[ch_idx],
                duration=scan_duration_per_column,
                sample_rate=sample_rate,
                amplitude=avg_intensity * master_volume
            )
            audio_channels_data[ch_idx].append(segment)

    final_audio_streams = []
    min_len_stream = float('inf')
    for ch_idx in range(3):
        if audio_channels_data[ch_idx]:
            stream = np.concatenate(audio_channels_data[ch_idx])
            final_audio_streams.append(stream)
            min_len_stream = min(min_len_stream, len(stream))
        else:
            total_samples = int(sample_rate * width * scan_duration_per_column)
            final_audio_streams.append(np.zeros(total_samples))
            min_len_stream = min(min_len_stream, total_samples)

    # Ensure all streams have the same length (due to potential floating point inaccuracies in linspace)
    if width > 0 : # Only if image has width
        for i in range(len(final_audio_streams)):
            if len(final_audio_streams[i]) > min_len_stream:
                final_audio_streams[i] = final_audio_streams[i][:min_len_stream]


    # --- Output Audio ---
    if not final_audio_streams or len(final_audio_streams[0]) == 0:
        print("No audio data generated (e.g., image width might be 0 or processed to 0). Cannot create WAV.")
        return

    if output_channels == 3:
        output_audio = np.stack(final_audio_streams, axis=-1)
    elif output_channels == 2:
        left_channel = final_audio_streams[0] + 0.707 * final_audio_streams[2]
        right_channel = final_audio_streams[1] + 0.707 * final_audio_streams[2]
        output_audio = np.stack((left_channel, right_channel), axis=-1)
    else:
        print(f"Unsupported number of output_channels: {output_channels}. Defaulting to 3.")
        output_audio = np.stack(final_audio_streams, axis=-1)

    max_amp = np.max(np.abs(output_audio))
    if max_amp > 0:
        output_audio_normalized = output_audio / max_amp
    else:
        output_audio_normalized = output_audio

    audio_to_write = np.int16(output_audio_normalized * 32767)

    try:
        wavfile.write(output_wav_path, sample_rate, audio_to_write)
        print(f"Successfully generated audio: {output_wav_path}")
        print(f"Audio duration: {len(output_audio) / sample_rate:.2f} seconds")
        # To play in Colab (optional, might not work for multi-channel directly in notebook)
        # from IPython.display import Audio
        # display(Audio(output_wav_path))
    except Exception as e:
        print(f"Error writing WAV file: {e}")

# ----------------------------------------------------
# Step 4: How to use in Colab
# ----------------------------------------------------
#
# 1. Upload your .tif file to your Colab session:
#    Click on the "Files" icon (folder icon) on the left sidebar.
#    Click the "Upload to session storage" button (upward arrow icon).
#    Select your .tif file.
#
#
#

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 19.9 MB/s eta 0:00:00


In [ ]:

my_tiff_file = "/content/EpH4-ECM_overlay+inibs_substr-ZO1+phaloidin-lrECM+DMSO-rep1-201124_img_8_z008_allchannels.tif" # <--- CHANGE THIS to your uploaded file name
output_audio_file = "sonified_lrECMmicroscopy.wav"
#
# Check if the file exists before processing (good practice)
import os
if os.path.exists(my_tiff_file):
     sonify_3channel_tiff(
         tiff_path=my_tiff_file,
         output_wav_path=output_audio_file,
         scan_duration_per_column=0.1,
         base_frequencies=(261.63, 329.63, 392.00), # C4, E4, G4 (C major chord)
         master_volume=0.7,
         output_channels=3 # or 2 for stereo
     )
#
#     # If you want to download the file after generation:
#     # from google.colab import files
#     # files.download(output_audio_file)
else:
    print(f"File not found: {my_tiff_file}. Please upload it first.")

Image loaded: 1392x1040 pixels, 3 channels.
Image dtype: uint16, min: 51, max: 4095
Successfully generated audio: sonified_lrECMmicroscopy.wav
Audio duration: 139.20 seconds
